In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import cv2
import os
from dbr import *
#from pyzbar import pyzbar
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from PIL import ImageEnhance
import tempfile
# from google.colab.patches import cv2_imshow
import shutil

In [4]:
from skimage.transform import hough_line, hough_line_peaks
from skimage.transform import rotate, resize
from skimage.feature import canny
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from scipy.stats import mode

ModuleNotFoundError: No module named 'skimage'

In [6]:
folder_path = # give folder path

In [ ]:
def remove_shadow(img):
  rgb_planes = cv2.split(img)

  result_planes = []
  result_norm_planes = []
  for plane in rgb_planes:
      dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
      bg_img = cv2.medianBlur(dilated_img, 21)
      diff_img = 255 - cv2.absdiff(plane, bg_img)
      norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
      result_planes.append(diff_img)
      result_norm_planes.append(norm_img)

  result = cv2.merge(result_planes)
  result_norm = cv2.merge(result_norm_planes)

  return result

In [ ]:
c = 0
for filename in os.listdir(folder_path):
  image_path = os.path.join(folder_path, filename)
  img = cv2.imread(image_path)
  #cv2_imshow(img)
  img = remove_shadow(img)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  ret, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
  contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
  contours2=[]
  cv2_imshow(gray)
  for cnt in contours:
      if(cv2.contourArea(cnt)>1500):
          contours2.append(cnt)
  coordinates = []
  for cnt in contours2:
      [x, y, w, h] = cv2.boundingRect(cnt)
      coordinates.append((x, y, w, h))
  for box in coordinates:
    x1= box[0]
    x2=(box[0]+box[2])
    y1=box[1]
    y2=(box[1]+box[3])
    cropped_box = gray[y1:y2, x1:x2]
    barcodes = pyzbar.decode(cropped_box)
    if len(barcodes) > 0:
      cv2_imshow(cropped_box)
      print("Barcode detected in", box)
      for barcode in barcodes:
          barcode_data = barcode.data.decode("utf-8")
          barcode_type = barcode.type
          print("Barcode Type:", barcode_type)
          print("Barcode Data:", barcode_data)
  print('moving to next image.......')

In [ ]:
c=0
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    if os.path.isfile(file_path) and any(file_path.endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
        image = cv2.imread(file_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        barcodes = pyzbar.decode(gray)

        if len(barcodes) > 0:
          c+=1
          for barcode in barcodes:
              barcode_data = barcode.data.decode("utf-8")
              barcode_type = barcode.type
              print("Barcode Type:", barcode_type)
              print("Barcode Data:", barcode_data)
        else:
          cv2_imshow(image)
    print('moving to next image.......')
print(c)

In [ ]:
c=0
k=0
BarcodeReader.init_license = #dbr license code
reader = BarcodeReader()
usable_folder = #
non_usable_folder = #
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and any(file_path.endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
      try:
        results = reader.decode_file(file_path)
        if results != None:
         for text_result in results:
          print("Barcode Format : " + text_result.barcode_format_string)
          print("Barcode Text : " + text_result.barcode_text)
          c+=1
          print(c)
        else:
          try:
            temp_path = enhance(file_path)
          except:
            print('image can not be enhanced')
            continue
          if os.path.isfile(temp_path) and any(temp_path.endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
            try:
              results = reader.decode_file(temp_path)
              if results != None:
                for text_result in results:
                  print("Barcode Format : " + text_result.barcode_format_string)
                  print("Barcode Text : " + text_result.barcode_text)
                  c+=1
                  print(c)
            except BarcodeReaderError as bre:
              print(bre)
      except BarcodeReaderError as bre:
        print(bre)
    k+=1
    print(k)
    print('moving to next image.......')
print(c)
del reader
os.remove(temp_path)

In [7]:
def enhance(file_path):
  temp_path = tempfile.mktemp(suffix=".jpg")
  img = Image.open(file_path)
  curr_bri = ImageEnhance.Brightness(img)
  new_bri = 1.2
  img = curr_bri.enhance(new_bri)
  curr_col = ImageEnhance.Color(img)
  new_col = 2.5
  img = curr_col.enhance(new_col)
  curr_con = ImageEnhance.Contrast(img)
  new_con = 1.2
  img= curr_con.enhance(new_con)
  img = img.convert('RGB')
  img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
  cv2.imwrite(temp_path, img)
  return temp_path

## extra

In [2]:
df.columns

Index(['Tracking Number', 'Tracking Status', 'Courier Company',
       'Courier Service', 'POD Link', 'Usable / Non-usable'],
      dtype='object')

In [3]:
url_links = list(df['POD Link'])

In [9]:
image_names = []
for link in url_links:
    name = link.split('/')[-1]
    ext = name[-4:]
    if ext not in ['.png', '.jpg', 'jpeg']:
        name = name + '.jpg'
    image_names.append(name)

In [10]:
import os

In [12]:
usable_folder = #
all_usable_images = os.listdir(usable_folder)        

In [14]:
for i, img_name in enumerate(image_names):
    if img_name in all_usable_images:
        df.loc[i, 'Usable / Non-usable'] = 'Usable'
    else:
        df.loc[i, 'Usable / Non-usable'] = 'Non-usable' 

In [15]:
df.head()

,Tracking Number,Tracking Status,Courier Company,Courier Service,POD Link,Usable / Non-usable
0,C701088203,Delivered,sahara_express,NaN,http://demo.saharaexpress.com/mobile_images/C7...,Usable
1,C701088203,Delivered,sahara_express,NaN,http://demo.saharaexpress.com/mobile_images/C7...,Usable
2,C701088203,Delivered,sahara_express,NaN,http://demo.saharaexpress.com/mobile_images/C7...,Usable
3,C701088203,Delivered,sahara_express,NaN,http://demo.saharaexpress.com/mobile_images/C7...,Usable
4,2020048233,Delivered,unique5pl,ROAD,https://unique5pl.com/u_pod/upload_pod/2020048...,Non-usable


In [16]:
df.to_excel(#)

In [ ]:
c=0
k=0
BarcodeReader.init_license(#dbr license code)
reader = BarcodeReader()
usable_folder = ##
non_usable_folder = ##
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and any(file_path.endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
      try:
        results = reader.decode_file(file_path)
        if results != None:
         for text_result in results:
          print("Barcode Format : " + text_result.barcode_format_string)
          print("Barcode Text : " + text_result.barcode_text)
          c+=1
          print(c)
        else:
          try:
            temp_path = enhance(file_path)
          except:
            print('image can not be enhanced')
            continue
          if os.path.isfile(temp_path) and any(temp_path.endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
            try:
              results = reader.decode_file(temp_path)
              if results != None:
                for text_result in results:
                  print("Barcode Format : " + text_result.barcode_format_string)
                  print("Barcode Text : " + text_result.barcode_text)
                  c+=1
                  print(c)
            except BarcodeReaderError as bre:
              print(bre)
      except BarcodeReaderError as bre:
        print(bre)
    k+=1
    print(k)
    print('moving to next image.......')
print(c)
del reader
os.remove(temp_path)